## AutoEncoder Wind Farm A


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GaussianNoise
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
import os

# Custom EarlyStopping callback monitoring L2 norm reconstruction error on validation set
class L2NormEarlyStopping(tf.keras.callbacks.Callback):
    def __init__(self, validation_data, patience=3):
        super().__init__()
        self.X_val = validation_data[0]
        self.y_val = validation_data[1]  # same as X_val for AE
        self.patience = patience
        self.best_weights = None
        self.best_l2_norm = np.Inf
        self.wait = 0
        self.stopped_epoch = 0

    def on_epoch_end(self, epoch, logs=None):
        val_pred = self.model.predict(self.X_val, verbose=0)
        l2_norm_error = np.linalg.norm(self.X_val - val_pred)
        print(f"Epoch {epoch+1}: Validation L2 norm reconstruction error = {l2_norm_error:.6f}")

        if l2_norm_error < self.best_l2_norm:
            self.best_l2_norm = l2_norm_error
            self.wait = 0
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print(f"Early stopping triggered at epoch {self.stopped_epoch + 1}. Restoring best model weights.")
                self.model.set_weights(self.best_weights)

# Dataset list for WindFarm A
dataset_ids = [0,3,10,13,14,17,22,24,25,26,38,40,42,45,51,68,69,71,72,73,84,92]



save_dir = r"D:\Master Thesis New Data Set\AutoEncoder\Wind Farm A\Power_Final"
os.makedirs(save_dir, exist_ok=True)

for dataset_id in dataset_ids:
    print(f"\n=== Processing WindFarm A Dataset {dataset_id} ===")

    df = pd.read_csv(
        rf"D:\Master Thesis New Data Set\Final DataSet\Wind Farm A Ultimate\{dataset_id}_WindFarm_A.csv",
        delimiter=','
    )

    df['time_stamp'] = pd.to_datetime(df['time_stamp'])
    df['train_test'] = df['train_test'].astype(str).str.strip().str.lower()

    # Ensure positive power values
    df['sensor_50'] = df['sensor_50'].apply(lambda x: max(x, 0))

    # Step 1: Filter normal operation for rated power determination
    df_filtered = df[
        (df['sensor_5_avg'] <= 2) & 
        (df['sensor_5_avg'] >= -5) & 
        (df['sensor_2_avg'] >= -10) & 
        (df['sensor_2_avg'] <= 10) & 
        (df['status_type_id'] == 0) & 
        (df['train_test'] == 'train')
    ].dropna(subset=['wind_speed_3_avg', 'sensor_50'])

    df_sorted = df_filtered.sort_values(by='wind_speed_3_avg')
    max_power = df_sorted['sensor_50'].max()

    # Rated band: power within 98%–102% of max power and wind speed between 10.7 and 12
    rated_band = df_sorted[
        (df_sorted['sensor_50'] >= max_power * 0.98) & 
        (df_sorted['sensor_50'] <= max_power * 1.02) & 
        (df_sorted['wind_speed_3_avg'] >= 10) & 
        (df_sorted['wind_speed_3_avg'] <= 12)
    ]

    if rated_band.empty:
        print(f"⚠️ Rated band empty for Dataset {dataset_id}. Skipping this dataset.")
        continue

    rated_speed = rated_band['wind_speed_3_avg'].min()
    rated_power = max_power
    power_threshold = 0.04 * rated_power
    print(f"Rated power: {rated_power:.4f}, Using power threshold: {power_threshold:.4f} (4% of rated power)")

    # Enhanced plausibility filter (Fraunhofer + your power threshold)
    normal_df = df[
        (df['status_type_id'] == 0) &
        (df['train_test'] == 'train') &
        ~((df['wind_speed_3_avg'] >= 3) & 
          (df['wind_speed_3_avg'] <= 25) & 
          (df['sensor_50'] <= power_threshold))  # Ignore low power within normal wind speed range
    ].copy()

    exclude_cols = ['time_stamp', 'train_test', 'id', 'asset_id']
    feature_cols = [c for c in df.columns if c not in exclude_cols]

    X = normal_df[feature_cols].apply(pd.to_numeric, errors='coerce').fillna(0).values

    if X.shape[0] < 50:
        print(f"⚠️ Skipping dataset {dataset_id} due to insufficient training data after filtering.")
        continue

    X_train, X_val = train_test_split(X, test_size=0.25, random_state=42)

    input_dim = X_train.shape[1]

    autoencoder = Sequential([
        GaussianNoise(0.06, input_shape=(input_dim,)),
        Dense(44, activation='relu'),
        Dense(25, activation='relu'),
        Dense(4, activation='relu'),
        Dense(25, activation='relu'),
        Dense(44, activation='relu'),
        Dense(input_dim, activation='linear')
    ])

    autoencoder.compile(optimizer=Adam(learning_rate=0.0018), loss='mse')

    l2_early_stop = L2NormEarlyStopping(validation_data=(X_val, X_val), patience=3)

    autoencoder.fit(
        X_train, X_train,
        validation_data=(X_val, X_val),
        epochs=200,
        batch_size=64,
        callbacks=[l2_early_stop],
        verbose=1
    )

    # Validation reconstruction errors (L2 norm per sample)
    reconstructions_val = autoencoder.predict(X_val)
    reconstruction_errors_val = np.linalg.norm(X_val - reconstructions_val, axis=1)

    # NN regressor threshold model
    nn_regressor = MLPRegressor(hidden_layer_sizes=(23,), activation='relu', max_iter=300, random_state=42)
    nn_regressor.fit(X_val, reconstruction_errors_val)

    predict_df = df[df['train_test'] == 'prediction'].copy()
    if predict_df.empty:
        print(f"⚠️ No prediction data for dataset {dataset_id}, skipping prediction.")
        continue

    X_predict = predict_df[feature_cols].apply(pd.to_numeric, errors='coerce').fillna(0).values

    reconstructions_pred = autoencoder.predict(X_predict)
    actual_errors = np.linalg.norm(X_predict - reconstructions_pred, axis=1)
    expected_errors = nn_regressor.predict(X_predict)

    gamma = 0.344  # Fraunhofer gamma for Wind Farm A
    is_anomaly = (actual_errors > expected_errors + gamma).astype(int)

    predict_df['predicted_status_type_id'] = np.where(is_anomaly == 1, 1, 0)

    save_path = os.path.join(save_dir, f"{dataset_id}_WindFarmA_status_predictions.csv")
    predict_df.to_csv(save_path, index=False)

    print(f"✅ Saved WindFarm A prediction for Dataset {dataset_id} — Anomalies detected: {sum(is_anomaly)}")

print("\n🎉 All WindFarm A datasets processed. AE-NBM predictions complete.")



=== Processing WindFarm A Dataset 0 ===
Rated power: 333090.0000, Using power threshold: 13323.6000 (10% of rated power)


C:\Users\49157\miniconda3\lib\site-packages\keras\src\layers\regularization\gaussian_noise.py:29: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/200
322/339 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 187385888.0000Epoch 1: Validation L2 norm reconstruction error = 64878.302279
339/339 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 180175504.0000 - val_loss: 7104.7241
Epoch 2/200
317/339 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 40816.8828Epoch 2: Validation L2 norm reconstruction error = 59918.637964
339/339 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 41575.4766 - val_loss: 6059.9946
Epoch 3/200
337/339 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 27512.6523Epoch 3: Validation L2 norm reconstruction error = 61085.303006
339/339 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 27598.5137 - val_loss: 6298.2778
Epoch 4/200
320/339 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 33396.5977Epoch 4: Validation L2 norm reconstruction error = 204209.838449
339/339 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 33226.7344 - val_loss: 70388.4766
Epoch 5/200
335/339 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 39367.3633Epoch 5: Validation L2 norm reconstruction error = 

C:\Users\49157\miniconda3\lib\site-packages\keras\src\layers\regularization\gaussian_noise.py:29: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


358/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 119226344.0000Epoch 1: Validation L2 norm reconstruction error = 59994.431511
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 114982872.0000 - val_loss: 5492.9663
Epoch 2/200
374/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 13089.2529Epoch 2: Validation L2 norm reconstruction error = 52117.755198
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 13092.5371 - val_loss: 4145.3047
Epoch 3/200
356/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6874.1719Epoch 3: Validation L2 norm reconstruction error = 62760.969990
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 6905.0200 - val_loss: 6011.2427
Epoch 4/200
364/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 13406.2607Epoch 4: Validation L2 norm reconstruction error = 50806.728606
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 13505.9453 - val_loss: 3939.3760
Epoch 5/200
374/375 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5443.1973Epoch 5: Validation L2 norm reconstruction error = 60230.688868
375/

...

326/349 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 171082832.0000Epoch 1: Validation L2 norm reconstruction error = 97524.967068
349/349 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 162429344.0000 - val_loss: 15615.1406
Epoch 2/200
338/349 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 27611.6445Epoch 2: Validation L2 norm reconstruction error = 68370.825531
349/349 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 27588.1562 - val_loss: 7674.6040
Epoch 3/200
345/349 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 27448.7578Epoch 3: Validation L2 norm reconstruction error = 65327.230868
349/349 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 27416.7871 - val_loss: 7006.5244
Epoch 4/200
333/349 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 35735.5547Epoch 4: Validation L2 norm reconstruction error = 104981.082064
349/349 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 35327.6836 - val_loss: 18094.0703
Epoch 5/200
340/349 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 16953.7363Epoch 5: Validation L2 norm reconstruction error = 70988.08776

C:\Users\49157\miniconda3\lib\site-packages\keras\src\layers\regularization\gaussian_noise.py:29: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


394/400 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 511697440.0000Epoch 1: Validation L2 norm reconstruction error = 1160721.957543
400/400 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 508283424.0000 - val_loss: 1929785.0000
Epoch 2/200
382/400 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1870446.2500Epoch 2: Validation L2 norm reconstruction error = 1160168.914477
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1870810.1250 - val_loss: 1927946.6250
Epoch 3/200
380/400 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1841780.2500Epoch 3: Validation L2 norm reconstruction error = 1157980.471718
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 1843593.7500 - val_loss: 1920679.7500
Epoch 4/200
390/400 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1850540.0000Epoch 4: Validation L2 norm reconstruction error = 1159514.270998
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 1851269.3750 - val_loss: 1925770.8750
Epoch 5/200
399/400 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1899691.3750Epoch 5: Validation L2 norm rec

C:\Users\49157\miniconda3\lib\site-packages\keras\src\layers\regularization\gaussian_noise.py:29: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


356/365 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 207055600.0000Epoch 1: Validation L2 norm reconstruction error = 360522.176562
365/365 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 203251488.0000 - val_loss: 203684.9531
Epoch 2/200
345/365 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 107480.5234Epoch 2: Validation L2 norm reconstruction error = 336697.609768
365/365 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 107806.4531 - val_loss: 177654.0156
Epoch 3/200
360/365 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 43215.7969Epoch 3: Validation L2 norm reconstruction error = 318702.064274
365/365 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 44131.8320 - val_loss: 159171.2969
Epoch 4/200
350/365 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 84768.3984Epoch 4: Validation L2 norm reconstruction error = 327033.741691
365/365 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 84875.9531 - val_loss: 167602.3438
Epoch 5/200
354/365 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 94228.9766Epoch 5: Validation L2 norm reconstruction error = 